<center> <h2> Avengers Tower is Relocating to Boston! </h2> </center>

<br />

<center><img src = "https://vignette.wikia.nocookie.net/marvelcinematicuniverse/images/0/09/Avengers_Tower_AoU_cropped.png/revision/latest/scale-to-width-down/289?cb=20170328174622" width = 200/></center>

You might have been led to believe that **Avengers Tower** has relocated to upstate New York. Don't worry; you're not alone. Tony Stark is smart... ;)

It turns out the tower is in fact relocating to Boston (well, it should have been constructed here in the first place anyways. I mean NYC?). :) 

As his entire automobile fleet is being shipped to Boston, Stark needs your help in learning more about the used car market in Boston, especially given the increasing prices due to the pandemic. It turns out he does need a car and will need to purchase a used car.

As usual, Stark made it sound like he wants you to practice your web scraping skills by helping him scrape used car listings from Craigslist (CL). That said, your cynical view is that he is trying to make you do his job! And you're probably right.

<hr>

## Getting started

* You will need to scrape the used cars for sale list from https://boston.craigslist.org.
* It is probably a very good idea to familiarize yourself with the backend structure of the website before you attempt to scrape data from it! Go ahead and explore the site and inspect the HTML code.


**Important advice:**
* Focus on each step of web scraping one at a time. If you try to do everything at once, it may get ugly.
* Write code for the HW problems you have, not the ones you wish you had.

In [ ]:
"https://boston.craigslist.org"

'https://boston.craigslist.org'

### Question 1 
Although you will be scraping the https://boston.craigslist.org URL, Tony would like you to write a function that takes two arguments, CL homepage link and a category name, and returns the URL of the category page.

* The idea is that because CL is available across the country, this function could be used to scrape data from CL for other cities as well.
* On CL, cars for sale are posted under "cars+trucks" category. If you wanted to scrape data from that page, you would need to access https://boston.craigslist.org/d/cars-trucks/search/cta
* Because CL uses the same URL structure across all cities in the country, writing a function might be useful!
* Besides, the backend URL conventions on CL do change from time to time, but the category names are rather constant. Thus, using a hard-coded URL could lead to issues down the road (if the page structure should get updated)


* Here are some sample function calls:

In [ ]:
# get_url_from_cl("https://boston.craigslist.org", "cars+trucks")
# this returns the URL of the "cars+trucks" page on boston.craigslist.com

In [ ]:
# get_url_from_cl("https://newyork.craigslist.org", "cars+trucks")
# this returns the URL of the "cars+trucks" page on newyork.craigslist.com

**Hint:**
* your function should take the URL in the first argument and the category name in the second argument.
* search for the URL of the second argument on the CL homepage provided in the first argument.
* and then return the URL for the category provided in the second argument.
* manually concatenating the two arguments based on the above output is not acceptable.

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as urllib
from urllib.error import URLError, HTTPError

In [ ]:
def get_url_from_cl(cl_url, cl_label):
    """ 
        gets craigslist url based on location and category
            
        Args:
            cl_url (string): craigslist location url
            cl_label (string): category label
            
        Returns:
            string: full craigslist url for location and category

    """
    
    try:
        html = urllib.urlopen(cl_url)
        soup = BeautifulSoup(html.read())
        html.close()
    except URLError:
        return "invalid url"
    except HTTPError:
        return "request failed"
    
    # restrict tags to area with categories
    categories = soup.find("div", {"id": "center"})
    
    # search area of categories for proper url
    # find the a tag with span containing text which matches category label then extract link
    try:
        cat_url = categories.find(lambda x: x.name == "a" and x.span.text.strip() == cl_label).get("href").strip()
    except:
        return "category doesn't exist"
    
    # concatenate category hyperlink with original for full url
    return cl_url + cat_url
    

In [ ]:
get_url_from_cl("https://boston.craigslist.org", "cars+trucks")
# this returns the URL of the "cars+trucks" page on boston.craigslist.com

'https://boston.craigslist.org/d/cars-trucks/search/cta'

In [ ]:
get_url_from_cl("https://newyork.craigslist.org", "cars+trucks")
# this returns the URL of the "cars+trucks" page on newyork.craigslist.com

'https://newyork.craigslist.org/d/cars-trucks/search/cta'

### Question 2 
From the "cars+trucks" listings page, you will scrape 4 pieces of information about each post: **listing price of the user card**, **the title of the post**, **when the listing was posted**, and **the link to the post**, as shown below:

<center> <img src = "https://raw.githubusercontent.com/yildirimcaglar/yildirimcaglar.github.io/master/ds3000/CL_Car.png" width = 300 /> </src>

* The good thing is you only need to scrape data from the first page, which by default provides a list of about 120 listings. And this is enough for Stark.

* You will store your scraped data in a DataFrame. 

**Therefore, in this question, you are asked to define an *empty* DataFrame, df_cars, with four columns: Price, Title, Posting Date, and URL.**

For now, write a code snippet to simply define the empty dataframe as shown below.



In [ ]:
import pandas as pd
df_cars = pd.DataFrame(columns=["Price", "Title", "Posting Date", "URL"])

In [ ]:
#when executed, an empty DataFrame with the above 4 columns should be displayed
df_cars

,Price,Title,Posting Date,URL


### Question 3 
Write a function, open_page_and_return_soup, that takes two arguments: the link to the CL homepage to open and the category name. The function will then use this information to first extract the URL corresponding to that category on the homepage link provided. Needles to say, you should use the function you wrote above inside this function. Afterwards, the function should open the requested cars+trucks page, turn the page contents into a BeautifulSoup object, and return that BeautifulSoup object. 

Refer to the sample function call below. Pay attention to the data type.

In [ ]:
def open_page_and_return_soup(cl_url, cl_label):
    """ 
        gets soup of results page url based on craiglist url
            
        Args:
            cl_url (string): craigslist location url
            cl_label (string): category label
            
        Returns:
            bs4.BeautifulSoup: soup of page results

    """
    
    url = get_url_from_cl(cl_url, cl_label)
    if not url.startswith("https://"):
        # returns previous error string
        return url
    else:
        # open page into soup
        try:
            html = urllib.urlopen(url)
            soup = BeautifulSoup(html.read())
            html.close()
        except HTTPError:
            return "request failed"
        return soup

In [ ]:
soup = open_page_and_return_soup("https://boston.craigslist.org", "cars+trucks")

In [ ]:
type(soup)

bs4.BeautifulSoup

### Warning:
If you make multiple requests, CL will eventually block you! That's why you are being asked to create the BeautifulSoup object here and avoid executing the URL request repeatedly. Once created, **soup** will enable you to do want you want. Refrain from making multiple requests to CL! They won't like it. Part of the assignment involves you not getting blocked. It's a reality of web scraping.

### Question 4 
Write a code snippet that extracts from your BeautifulSoup object the aforementioned 4 pieces of information about each post: **listing price of the car**, **the title of the post**, **when the listing was posted**, and **the link to the post**.

For each listing, you should store these values in their corresponding columns of the df_cars DataFrame.

In [ ]:
# array to append all results
vehicles = []

# get a list of the results (rows of df)
vehicle_divs = soup.find_all("li", class_="result-row")

# get properties we are looking for (columns of df)
for vehicle in vehicle_divs:
    list_price = vehicle.find("span", "result-price").get_text().strip()
    title = vehicle.find("a", "result-title").get_text().strip()
    list_date = vehicle.find("time", "result-date").get_text().strip()
    post_url = vehicle.find("a", "result-title").get("href").strip()
    vehicle_dict = {
        "Price": list_price,
        "Title": title,
        "Posting Date": list_date,
        "URL": post_url
    }
    vehicles.append(vehicle_dict)

# add all results to df at once
df_cars = df_cars.append(vehicles)


When executed, the following line should display your DataFrame as follows:
* Note that the price values do not contain the $ sign.
* Some of these fields may not be available for all listings. Your code should take that into account and leave the field value empty ("") when a field is not available for the listing.
* Pay attention to the format of the posting date. There may be multiple formats in the HTML, but you should scrape the one displayed below.


Refer to the sample output below. Note that by the time you scrape the page, new postings will have been made. Thus, your listings will differ.

In [ ]:
df_cars

,Price,Title,Posting Date,URL
0,"$22,500",2013 Chevy Silverado 1500,Sep 18,https://boston.craigslist.org/nwb/cto/d/dracut...
1,$800,1995 Chevrolet C/K Pickup 1500 Regular Cab 4WD,Sep 18,https://boston.craigslist.org/gbs/ctd/d/bozema...
2,$99,2020 Hyundai Accent SE - BAD CREDIT OK!,Sep 18,https://boston.craigslist.org/nwb/ctd/d/salem-...
3,"$2,900",2004 Audi S4 Quattro AWD V-8,Sep 18,https://boston.craigslist.org/gbs/cto/d/pittsf...
4,"$12,495",2015 Toyota Corolla LE,Sep 18,https://boston.craigslist.org/gbs/ctd/d/malden...
...,...,...,...,...
115,"$5,995",2010 Hyundai Genesis v8 LOADED,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...
116,"$21,000","2013 FX4, Loaded",Sep 18,https://boston.craigslist.org/nos/cto/d/danver...
117,"$5,995",2004 Dodge Ram Pickup 2500 Laramie 4dr Quad Ca...,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...
118,"$6,995",2007 Chevrolet Silverado 1500 LT1 4dr Extended...,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...


### Question 5 
Being the pain in the neck he is, Tony would also like you to extract two other pieces of information from each listing's own page and add those to the dataframe. Specifically, for each car in the list, he would like you to retrieve the odometer value of the car as well as transmission information (whether it is manual, automatic, etc.). It turns out he cannot drive a stick shift. Both fields can only be accessed from each listing's own page, so the homepage isn't helpful here.

To get started, pick a car listing and open the posting in your browser. On the right-hand side, you will see the attributes of the car in gray boxes. Odometer and transmission information, when available, is shown there too. Study the HTML code for that portion of the screen!


* Your code snippet should look through the above BeautifulSoup object. 
    * **Hint:** Note that in Q4, you already got a list of the cars from the BeautifulSoup object, so you can use that here.
* For each listing, go to the listing's page. Once you have accessed the page information, remember to close the connection! Otherwise, you may get blocked.
* Retrieve the odometer and transmission information, if available, and store them
    * If these fields are not available, use empty ("") placeholder values for these fields.
* As we discussed, before moving on to the next car in the list of cars, you might find it useful to pause the program for 5-10 seconds (randomly determined). This will take some time, of course. However, CL is really snippy about people crawling their site. You will get blocked, otherwise, and you really cannot afford that if you want to finish the assignment on time. :)
    * If you do get blocked, try another computer or Google Colab on a lab computer on campus
    
Once you have this information for all cars, update the df_cars dataframe with the odometer and transmission information as shown in the sample output below.

In [ ]:
import time
import random
import numpy as np

In [ ]:
# fixed number of listings to optimize np arrays
num_list = df_cars.shape[0]

# np arrays to store new column info
# use '' as default
odometers = np.zeros(num_list, np.dtype("U10"))
transmissions = np.zeros(num_list, np.dtype("U20"))

# get new column info for each listing
for i in range(num_list):
    
    # provide indication for user that we are still scraping dataa
    print(f"Fetching car {i + 1} of {num_list}...")
    
    # get row
    vehicle_series = df_cars.iloc[i]
    
    # wait after first request so we don't get blocked
    if i != 0:
        time.sleep(random.randint(5,10))
        
    # retrieve page data in soup
    html = urllib.urlopen(vehicle_series["URL"])
    soup = BeautifulSoup(html.read())
    html.close()
    
    # get attributes on cl sidebar
    attributes = soup.find_all("p", "attrgroup")
    for all_attr in attributes:
        for attr in all_attr:
            # find attributes with span
            if attr.name == "span":
                attr_text = attr.get_text().strip()
                # check span text to see if it matches columns we need
                # extract data if it fits our model, else continue loop
                if attr_text.startswith("odometer:"):
                    try:
                        odometers[i] = attr_text.split(":")[1].strip()
                    except:
                        continue
                elif attr_text.startswith("transmission:"):
                    try:
                        transmissions[i] = attr_text.split(":")[1].strip()
                    except: continue

# indicate we are done scraping data
print("Done!")

# add new columns to df
df_cars["Odometer"] = odometers
df_cars["Transmission"] = transmissions
    

Fetching car 1 of 120...
Fetching car 2 of 120...
Fetching car 3 of 120...
Fetching car 4 of 120...
Fetching car 5 of 120...
Fetching car 6 of 120...
Fetching car 7 of 120...
Fetching car 8 of 120...
Fetching car 9 of 120...
Fetching car 10 of 120...
Fetching car 11 of 120...
Fetching car 12 of 120...
Fetching car 13 of 120...
Fetching car 14 of 120...
Fetching car 15 of 120...
Fetching car 16 of 120...
Fetching car 17 of 120...
Fetching car 18 of 120...
Fetching car 19 of 120...
Fetching car 20 of 120...
Fetching car 21 of 120...
Fetching car 22 of 120...
Fetching car 23 of 120...
Fetching car 24 of 120...
Fetching car 25 of 120...
Fetching car 26 of 120...
Fetching car 27 of 120...
Fetching car 28 of 120...
Fetching car 29 of 120...
Fetching car 30 of 120...
Fetching car 31 of 120...
Fetching car 32 of 120...
Fetching car 33 of 120...
Fetching car 34 of 120...
Fetching car 35 of 120...
Fetching car 36 of 120...
Fetching car 37 of 120...
Fetching car 38 of 120...
Fetching car 39 of 12

In [ ]:
df_cars

,Price,Title,Posting Date,URL,Odometer,Transmission
0,"$22,500",2013 Chevy Silverado 1500,Sep 18,https://boston.craigslist.org/nwb/cto/d/dracut...,94000,automatic
1,$800,1995 Chevrolet C/K Pickup 1500 Regular Cab 4WD,Sep 18,https://boston.craigslist.org/gbs/ctd/d/bozema...,127190,automatic
2,$99,2020 Hyundai Accent SE - BAD CREDIT OK!,Sep 18,https://boston.craigslist.org/nwb/ctd/d/salem-...,14216,automatic
3,"$2,900",2004 Audi S4 Quattro AWD V-8,Sep 18,https://boston.craigslist.org/gbs/cto/d/pittsf...,140000,automatic
4,"$12,495",2015 Toyota Corolla LE,Sep 18,https://boston.craigslist.org/gbs/ctd/d/malden...,127160,automatic
...,...,...,...,...,...,...
115,"$5,995",2010 Hyundai Genesis v8 LOADED,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...,161531,automatic
116,"$21,000","2013 FX4, Loaded",Sep 18,https://boston.craigslist.org/nos/cto/d/danver...,118000,automatic
117,"$5,995",2004 Dodge Ram Pickup 2500 Laramie 4dr Quad Ca...,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...,188868,automatic
118,"$6,995",2007 Chevrolet Silverado 1500 LT1 4dr Extended...,Sep 18,https://boston.craigslist.org/gbs/ctd/d/east-d...,140154,automatic
